# 说明文档

参与人员：
徐瑾涵 2021104247
余鼎成 2021104246
梁浩原 2021104515
邓博予 2021104065

github: https://github.com/ndce1041/Stock-Buying-Decision-Support-System/tree/master



此项目为：针对在当前时间是否对某只股票进行买入操作的决策支持系统，

### 人机交互界面
在项目中使用网页作为人机交互界面，提供详尽的参数操作和美观的数据可视化。

1. part1 训练集数据预览
将模型使用的训练集以形象可交互的形式绘制，方便使用者自己对结果的判断
<img src="html1.png">

2. part2 模型主要操作面板
提供了模型的选择和快捷的结果显示，右侧可变更需要预测的股票
<img src="html2.png">

3. part3 模型细节参数调整面板
提供各个模型详细的参数修改和结果预览，右侧的模型预览会显示此模型预测的后20天的涨跌情况，左侧最下方会显示在指定比例测试集下的误差情（注意：由于模型自身数学原理不同，显示的信息各有差异，有些模型并不能预测出准确数字）
<img src="html3.png">

同时对于模型的细节参数，我们添加了简要的参数解释，提供下拉框保证参数正确性
<img src="html4.png">



### 数据库
数据库使用第三方远程数据库，可获取最新和大量历史数据，为模型预测提供数据支持

### 模型库
模型库目前有5款预测模型，采用统一的接口标准和输出标准，每次预测会使用四年的数据，详细介绍如下

## SVM模型介绍

支持向量机（Support Vector Machine, SVM）是一类按监督学习（supervised learning）方式对数据进行二元分类的广义线性分类器（generalized linear classifier），在股市预测问题中，因为股市的状况可以近似地分类成“涨”、“跌”两种状态，因此可以考虑使用SVM进行预测。

SVM模型的基本思想是寻找一个超平面，使得正负样本的间隔（即样本到超平面的距离）最大。超平面即决策边界。

决策边界可以由

$$
x^Tw+b=-1,x^Tw+b=0,x^Tw+b=1
$$

表示。而算法的目标就是寻找一组直线的参数$w$和$b$，使得分类间隔取最大值。目标函数：

$$
\max_{w,b} {{2}\over{\Vert w \Vert}}
$$

约束条件：

$$
y_i(x_i^Tw+b) \geq 1
$$

而在大多数情况下由于噪音的存在，直接采用以上公式可能导致过拟合等问题，因此考虑采用**松弛变量**，对每个样本点赋予一个松弛变量的值：如果该点落在最大边缘超平面正确的一侧，则松弛变量$\xi$ = 0；否则，松弛变量的值等于该点到最大边缘超平面的距离。由此，目标函数变为：

$$
\min_{w,b,\xi} \left( {{\Vert w \Vert^2}\over{2}}+C\sum^{n}_{i=1}\xi_i \right)
$$

约束条件变为：

$$
y_i(x_i^Tw+b) \geq 1-\xi_i,\quad\xi_i\geq0
$$

目标函数中的$C$为惩罚系数，调整惩罚系数可以调控松弛变量的松弛程度。

而对于预测问题，对分类问题稍加改动就可以得到如下：

$$
\min_{w,b,\xi,\xi_i^*} \left( {{\Vert w \Vert^2}\over{2}}+C\sum^{n}_{i=1}(\xi_i+\xi_i^*) \right)
$$

$$
-\xi-\xi_i^*\leq y_i(x_i^Tw+b) \leq \xi+\xi_i^*,\quad\xi,\xi_i^*\geq0
$$

在回归问题总，SVM的目标是找到一个函数，使得该函数预测的值与实际值之间的误差最小，并且该函数的复杂度（通常是函数的平滑度）也要尽可能小。这就是所谓的结构风险最小化原则。SVM使用一种叫做核技巧（kernel trick）的方法来处理非线性问题。核技巧可以通过非线性映射将原始特征空间映射到一个更高维的空间，使得在这个高维空间中，样本数据变得线性可分。这使得SVM可以处理各种复杂的非线性问题。

SVM的关键问题在于核函数的选取，而股市问题明显为非线性可分数据集，一般适合采用rbf或多项式核。对于股市问题，由于现有数据仅有收盘价，缺少其他因子作为特征值，因此预测效果并不好。


## 移动平均线模型

简单移动平均（SMA）模型是用一组最近的实际数据值来预测未来数据的一种常用方法，当预测数据既不快速增长也不快速下降，且不存在季节性因素时，移动平均法能有效消除预测中的随机波动。它的基本思想是，预测的未来某一天的值是其前N天数据的平均值。例如未来第1天的数据可以为前30天数据的平均值，而未来第2天的数据为前29天和未来第一天数据的平均值。实际使用时，可以通过对N的不同设置以达到股市分析的目的。例如，当短期平均线（5日线）上穿较长期平均线（30日，60日），显示短线价格走强，策略可倾向做多；相反，当短期平均线（5日线）下穿较长期平均线（30日，60日），显示短线价格走弱，策略可倾向做空。因此，该模型可以对决策者提供基本决策支持。

## **LSTM**

### 0. ***RNN***

> 循环神经网络（Recurrent Neural Network, RNN）是一类以序列（sequence）数据为输入，在序列的演进方向进行递归（recursion）且所有节点（循环单元）按链式连接的递归神经网络（recursive neural network）

#### RNN的结构
神经网络假设所有的输入是相互独立的，对于某些任务来说这不是一个好的假设。比如你想预测一个句子的下一个词，知道之前的词是有帮助的。

而RNN的特点是利用时序的信息，RNN被称为循环的(recurrent)原因就是它会对一个序列的每一个元素执行同样的操作，并且之后的输出依赖于之前的计算。我们可以认为RNN有一些“记忆”能力，它能捕获之前计算过的一些信息。理论上RNN能够利用任意长序列的信息，但是实际中它能记忆的长度是有限的。

<img src="rnn展开图.jpeg" alt="RNN展开图" style="width:50%">

上图显示了怎么把一个RNN展开成一个完整的网络

### 1. **LSTM**

>长短期记忆网络（LSTM，Long Short-Term Memory）是一种时间循环神经网络(RNN)的变体

#### LSTM的结构
LSTM是一种特殊的RNN网络，它使用门(Gate)的机制来解决长距离依赖的问题。主要是为了解决长序列训练过程中的梯度消失和梯度爆炸问题。
>LSTM出现的原因
传统RNN的激活函数用的是sigmoid，它会将+∞～-∞之间的输入压缩到0～1之间。当input的值更新时，output会有很小的更新。
又因为上一层的输出将作为后一层的输入，而输出经过sigmoid之后更新速率会逐步衰减，直到输出层才会有微乎其微的更新。
从数学的角度出发，sigmoid函数在０的位置取得最大值１／４，当我们使用均值为0，方差为１的高斯分布初始化参数ｗ，有｜ｗ｜＜１。
随着网络层数的加深，ｗ的变化幅度会呈１／４的指数衰减，使得最后学不到东西，造成梯度弥散。
同样，当|w|>1,w变化幅度会指数递增，最后造成梯度爆炸。

所有的RNN都是如下图所示的结构，把RNN看成一个黑盒子的话，它会有一个“隐状态”来“记忆”一些重要的信息。当前时刻的输出除了受当前输入影响之外，也受这个“隐状态”影响。并且在这个时刻结束时，除了输出之外，这个“隐状态”的内容也会发生变化——它“记忆”了新的信息同时有“遗忘”了一些旧的信息。

<img src="rnn结构.png" alt="RNN结构图" style="width:50%">

LSTM也是这样的结构，但相比于原始的RNN，它的内部结构更加复杂。

普通的RNN就是一个全连接的层，而LSTM有四个用于控制”记忆”和运算的门，如下图所示。

<img src="lstm结构.png" alt="LSTM结构图" style="width:50%">

<img src="lstm符号说明.png" alt="符号说明" style="width:50%">

> 每个黄色方框表示一个神经网络层，由权值，偏置以及激活函数组成；每个粉色圆圈表示元素级别操作；箭头表示向量流向；相交的箭头表示向量的拼接；分叉的箭头表示向量的复制。

#### LSTM核心思想

相比于原始的RNN的隐层(hidden state)， LSTM增加了一个细胞状态(cell state)

我们可以先把中间的操作遮起来，看一下LSTM在t时刻的输入与输出，首先，输入有三个：细胞状态C_t-1，隐层状态h_t-1,t时刻输入向量Xt，而输出有两个：细胞状态C_t，隐层状态h_t，其中h_t 还作为t时刻的输出。LSTMs的核心是细胞状态，用贯穿细胞的水平线表示。细胞状态像传送带一样。它贯穿整个细胞却只有很少的分支，这样能保证信息不变的流过整个RNNs。细胞状态如下图所示：

<img src="lstm细胞状态.png" alt="LSTM细胞状态图" style="width:50%">

当然如果LSTM只是原封不动的保存之前的记忆，那就没有太多价值，它还必须根据需要，能够增加新的记忆同时擦除旧的无用的记忆。LSTM是通过一种叫作门的机制来控制怎么擦除旧记忆写入新记忆的。门能够有选择性的决定让哪些信息通过。其实门的结构很简单，就是一个sigmoid层和一个点乘操作的组合。如下图所示

<img src="lstmgate.png" alt="LSTM门结构图" >

前面提到LSTM由三个门来控制细胞状态，这三个门分别称为忘记门、输入门和输出门。下面一个一个的来讲述。
#### LSTM的门结构
##### 1. 忘记门
LSTM的第一步就是决定细胞状态需要丢弃哪些信息。这部分操作是通过一个称为忘记门的sigmoid单元来处理的。它通过查看Xt和ht-1信息来输出一个0-1之间的向量，该向量里面的0-1值表示细胞状态Ct-1中的哪些信息保留或丢弃多少。0表示不保留，1表示都保留。忘记门如下图所示。

<img src="lstm3-focus-f.png" alt="LSTM忘记门图" style="width:50%">

首先说一下[ h t − 1 , x t ] 这个东西就代表把两个向量连接起来（操作与numpy.concatenate相同）

##### 2. 输入门

下一步是决定给细胞状态添加哪些新的信息。这一步又分为两个步骤，首先，利用ht-1和Xt通过一个称为输入门的操作来决定更新哪些信息。然后利用ht-1和Xt通过一个tanh层得到新的候选细胞信息Ct~，这些信息可能会被更新到细胞信息中。这两步描述如下图所示。

<img src="lstm3-focus-i.png" alt="LSTM输入门图"style="width:50%">

下面将更新旧的细胞信息Ct-1，变为新的细胞信息Ct。更新的规则就是通过忘记门选择忘记旧细胞信息的一部分，通过输入门选择添加候选细胞信息Ct~的一部分得到新的细胞信息Ct。更新操作如下图所示

<img src="lstm3-focus-C.png" alt="LSTM细胞信息更新图"style="width:50%">


#### 3. 输出门

更新完细胞状态后需要根据输入的ht-1和Xt来判断输出细胞的哪些状态特征，这里需要将输入经过一个称为输出门的sigmoid层得到判断条件，然后将细胞状态经过tanh层得到一个-1~1之间值的向量，该向量与输出门得到的判断条件相乘就得到了最终该RNN单元的输出。该步骤如下图所示：

<img src="lstm3-focus-o.png" alt="LSTM输出门图"style="width:50%">


## LSTM历史及应用
### 应用现状
* LSTM应用的领域包括：文本生成、机器翻译、语音识别、生成图像描述和视频标记等。

* 2009年, 应用LSTM搭建的神经网络模型赢得了ICDAR手写识别比赛冠军。

* 2015年以来，在机械故障诊断和预测领域，相关学者应用LSTM来处理机械设备的振动信号。

* 2016年, 谷歌公司应用LSTM来做语音识别和文字翻译，其中Google翻译用的就是一个7-8层的LSTM模型。

* 2016年, 苹果公司使用LSTM来优化Siri应用。

 

### 发展历史
* 1997年，Sepp Hochreiter 和 Jürgen Schmidhuber[1]提出了长短期记忆神经网络(LSTM)，有效解决了RNN难以解决的人为延长时间任务的问题，并解决了RNN容易出现梯度消失的问题。

* 1999年，Felix A. Gers等人[2]发现[1]中提出的LSTM在处理连续输入数据时，如果没有重置网络内部的状态，最终会导致网络崩溃。因此，他们在文献[1]基础上引入了遗忘门机制，使得LSTM能够重置自己的状态。

* 2000年，Felix A. Gers和Jiirgen Schmidhuber[3]发现，通过在LSTM内部状态单元内添加窥视孔（Peephole）连接，可以增强网络对输入序列之间细微特征的区分能力。

* 2005年，Alex Graves和Jürgen Schmidhuber[4]在文献[1] [2] [3]的基础上提出了一种双向长短期记忆神经网络（BLSTM），也称为vanilla LSTM，是当前应用最广泛的一种LSTM模型。

* 2005年-2015年期间，相关学者提出了多种LSTM变体模型，此处不多做描述。

* 2016年，Klaus Greff 等人[5]回顾了LSTM的发展历程，并比较分析了八种LSTM变体在语音识别、手写识别和弦音乐建模方面的能力，实验结果表明这些变体不能显著改进标准LSTM体系结构，并证明了遗忘门和输出激活功能是LSTM的关键组成部分。在这八种变体中，vanilla LSTM的综合表现能力最佳。另外，还探索了LSTM相关超参数的设定影响，实验结果表明学习率是最关键的超参数，其次是网络规模（网络层数和隐藏层单元数），而动量梯度等设置对最终结果影响不大。

[1] S. Hochreiter and J. Schmidhuber, “Long Short-Term Memory,” Neural Comput, vol. 9, no. 8, pp. 1735–1780, Nov. 1997.

[2] F. A. Gers, J. Schmidhuber, and F. A. Cummins, “Learning to Forget: Continual Prediction with LSTM,” Neural Comput., vol. 12, pp. 2451–2471, 2000.

[3] F. A. Gers and J. Schmidhuber, “Recurrent nets that time and count,” Proc. IEEE-INNS-ENNS Int. Jt. Conf. Neural Netw. IJCNN 2000 Neural Comput. New Chall. Perspect. New Millenn., vol. 3, pp. 189–194 vol.3, 2000.

[4] A. Graves and J. Schmidhuber, “Framewise phoneme classification with bidirectional LSTM and other neural network architectures,” Neural Netw., vol. 18, no. 5, pp. 602–610, Jul. 2005.

[5] K. Greff, R. K. Srivastava, J. Koutník, B. R. Steunebrink, and J. Schmidhuber, “LSTM: A Search Space Odyssey,” IEEE Trans. Neural Netw. Learn. Syst., vol. 28, no. 10, pp. 2222–2232, Oct. 2017.

[6] K. Cho et al., “Learning Phrase Representations using RNN Encoder-Decoder for Statistical Machine Translation,” ArXiv14061078 Cs Stat, Jun. 2014.

[7] J. Chung, C. Gulcehre, K. Cho, and Y. Bengio, “Empirical Evaluation of Gated Recurrent Neural Networks on Sequence Modeling,” ArXiv14123555 Cs, Dec. 2014.

## 自回归差分移动平均模型

1.ARIMA模型全称为自回归差分移动平均模型。主要通过自回归、差分和移动平均组合来用于捕捉时间序列中的趋势和季节性规律。
ARIMA模型的模型公式为：
<img src="arima_pic.png">
 
其中，公式前半部分为自回归部分，$ \varphi_{i} $为自回归系数；后半部分是滑动平均部分，  为滑动平均系数， 的表达式为 。

2.ARIMA模型具有以下几个优势：
1. 灵活性：ARIMA模型可以适用于各种时间序列数据，包括非平稳和季节性数据。
2. 可解释性：ARIMA模型的参数具有明确的统计意义，可以帮助我们理解时间序列数据的特征和趋势。
3. 预测能力：ARIMA模型可以用于预测未来的时间序列值，帮助我们做出合理的决策和规划。
4. 自动调整：ARIMA模型可以自动调整参数，找到最佳的模型拟合，减少了手动调整参数的工作量。
5. 可解释性：ARIMA模型的结果可以通过残差分析来评估模型的拟合程度和预测精度，帮助我们评估模型的可靠性。
总的来说，ARIMA模型是一种强大的时间序列分析工具，可以帮助我们理解和预测时间序列数据的行为。

3.传入数据与传出数据：
传入参数为训练集占总样本的比例，自回归项，差分项及移动平均项
自回归项表示当前观测值与过去观测值之间的关系。它表示当前观测值与前几个观测值的线性组合。
差分项表示对观测值进行差分运算的次数，用来消除时间序列的非平稳性，使其变为平稳序列。
移动平均项表示当前观测值与过去观测值的误差的线性组合。它表示当前观测值与前几个观测值的误差的加权平均。
这些参数一起构成了ARIMA模型（自回归差分移动平均模型），用于对时间序列数据进行建模和预测。
同时，通过训练模型，我们可以得到预测值与真实值的均方误差，进而判断模型是否能较好预测。

输出数据为均方误差，模型表现，预测未来二十天的收盘价，以及判断是否买入。

4.实验过程：
我们输入训练集比例为0.8，对最后百分之二十天数的股价进行预测，并通过预测值与真实值相比计算均方误差，得到结果为0.005，远小于1，可以说明模型拟合度较好，进而对接下来二十天的数据进行预测，发现其平均值小于最后一天的收盘价，进而我们做出决策：卖掉股票！

<img src="arima_pic2.png">


## 证券技术改良后的朴素法

1.基于朴素法的交易模型，结合了技术分析的概念。
在这个模型中，首先根据给定的训练集占比，计算支撑线和阻力线。支撑线是训练集中收盘价的最小值，而阻力线是训练集中收盘价的最大值。然后，根据最后一天的收盘价和成交量与支撑线、阻力线的关系，以及与前一天的成交量的比较，判断是否应该买入或卖出。如果最后一天的收盘价低于支撑线且成交量大于前一天的成交量，则输出1表示买入；如果最后一天的收盘价高于阻力线且成交量大于前一天的成交量，则输出0表示卖出；否则输出0表示不进行交易。
接下来，使用测试集进行测试，并计算预测值与实际值之间的均方误差。如果均方误差小于1，则输出1表示模型表现较好；否则输出0表示模型表现较差。
最后，预测接下来20天的数据，并计算这20天收盘价的平均值。如果平均值大于今天的收盘价，则输出1表示买入；否则输出0表示卖出。

2.整个模型的输入是一个数据集和训练集占比，输出是均方误差、模型表现、预测值和是否买入的结果。

3.该模型的优势在于结合了朴素法和技术分析的思想，通过计算支撑线和阻力
线，并比较最后一天的收盘价与支撑线、阻力线的关系，以及成交量的变化情况，来判断是否买入或卖出。
同时，该模型还使用了训练集和测试集进行测试，计算均方误差，以评估模型的表现。最后，该模型还能预测接下来20天的数据，并根据预测结果判断是否买入或卖出。通过综合考虑多个因素，该模型可以提供较为准确的买卖决策。

4.实验过程
我们使用了朴素法模型来进行股票价格的预测和交易决策。根据模型的表现和预测结果，我们可以评估模型的准确性，并根据预测结果来决定是否进行买入或卖出操作。输入训练集占比为0.8，得到均方误差为0.0528，模型表现较好，预测接下来二十天的值为0.9469，不符合条件，因此做出决策：卖掉股票！

使用库文件
* selectors
* sklearn 1.3.2
* statsmodels 0.14.0
* tensorflow 2.15.0
* keras 2.15.0
* pandas 2.1.0
* numpy 1.25.2
* tushare 1.2.89
